
# Silver – Tabela de Controle do CDF

Este notebook cria a **tabela de controle** usada pela camada Silver para
rastrear a **última versão do Change Data Feed (_commit_version)** já aplicada
de cada tabela Bronze do **call center**.

**O que este notebook faz:**
1. Cria (se necessário) o **schema Silver** no catálogo escolhido.
2. Cria (se necessário) a tabela de controle `__last_processed_version` com:
   - `source_table` (FQN da tabela Bronze)
   - `last_version` (BIGINT, default 0)
   - `last_run_at` (TIMESTAMP, default 1970-01-01)
   - *Constraint* de sanidade (`last_version >= 0`)
3. **Seed inicial** (idempotente) com as **3 fontes Bronze** do call center:
   - URA  
   - Atendimento humano (encaminhadas)  
   - Pesquisa de satisfação  

> **Por que precisamos disso?**  
> Na Silver (batch/trigger-once), lemos o CDF **a partir do último _commit_version**
> aplicado. Essa tabela de controle guarda esse marcador por **fonte**, garantindo
> **idempotência** e **reprocesso seguro**.



## Parâmetros editáveis
Ajuste abaixo o **catálogo**, os **schemas** e os **nomes das tabelas Bronze**.
Os FQNs serão montados automaticamente.


In [0]:

# ===================== PARÂMETROS EDITÁVEIS =====================
catalog_name   = "prd"
bronze_schema  = "b_dm_callcenter"
silver_schema  = "s_dm_callcenter"

# Tabelas Bronze (nomes conforme seu projeto)
bronze_ura_table   = "ura_once"
bronze_calls_tbl   = "calls_once"
bronze_surveys_tbl = "surveys_once"

# ===================== DERIVA FQNs =====================
BRONZE_URA_FQN     = f"{catalog_name}.{bronze_schema}.{bronze_ura_table}"
BRONZE_CALLS_FQN   = f"{catalog_name}.{bronze_schema}.{bronze_calls_tbl}"
BRONZE_SURVEYS_FQN = f"{catalog_name}.{bronze_schema}.{bronze_surveys_tbl}"
CTRL_TBL_FQN       = f"{catalog_name}.{silver_schema}.__last_processed_version"

print("Bronze URA:        ", BRONZE_URA_FQN)
print("Bronze Ligações:   ", BRONZE_CALLS_FQN)
print("Bronze Pesquisa:   ", BRONZE_SURVEYS_FQN)
print("Tabela de controle:", CTRL_TBL_FQN)


Bronze URA:         prd.b_dm_callcenter.ura_once
Bronze Ligações:    prd.b_dm_callcenter.calls_once
Bronze Pesquisa:    prd.b_dm_callcenter.surveys_once
Tabela de controle: prd.s_dm_callcenter.__last_processed_version



## Criar tabela de controle (se não existir)
Estrutura mínima + constraints e comentário:
- `source_table` (**NOT NULL**): FQN da tabela Bronze  
- `last_version` (**NOT NULL**): último `_commit_version` aplicado  
- `last_run_at` (**NOT NULL**): timestamp do último run  

In [0]:
spark.sql(f'''
CREATE TABLE IF NOT EXISTS {CTRL_TBL_FQN} (
  source_table STRING NOT NULL,
  last_version BIGINT NOT NULL,
  last_run_at  TIMESTAMP NOT NULL
)
USING DELTA
COMMENT "Controle de última versão (_commit_version) processada por fonte Bronze para consumo CDF na Silver."
''')


DataFrame[]


## Seed inicial (idempotente) das 3 fontes Bronze
Insere as três fontes se **ainda não existirem** na tabela de controle (não duplica).


In [0]:
spark.sql(f'''
MERGE INTO {CTRL_TBL_FQN} t
USING (
  SELECT * FROM VALUES
    ("{BRONZE_URA_FQN}",  0, TIMESTAMP"1970-01-01 00:00:00"),
    ("{BRONZE_CALLS_FQN}",  0, TIMESTAMP"1970-01-01 00:00:00"),
    ("{BRONZE_SURVEYS_FQN}", 0, TIMESTAMP"1970-01-01 00:00:00")
  AS v(source_table, last_version, last_run_at)
) s
ON t.source_table = s.source_table
WHEN NOT MATCHED THEN INSERT *
''')


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]


## 🔎 Validação rápida
Confirme que as **três fontes** estão registradas com `last_version = 0`.


In [0]:
spark.sql(f"SELECT * FROM {CTRL_TBL_FQN} ORDER BY source_table").display()


source_table,last_version,last_run_at
prd.b_dm_callcenter.calls_once,0,1970-01-01T00:00:00Z
prd.b_dm_callcenter.surveys_once,0,1970-01-01T00:00:00Z
prd.b_dm_callcenter.ura_once,0,1970-01-01T00:00:00Z



## Reset (quando necessário)
Para recomputar do zero uma fonte específica, rode no **%sql** ou via `spark.sql`:
```sql
UPDATE {catalog}.{schema}.__last_processed_version
SET last_version = 0, last_run_at = current_timestamp()
WHERE source_table = '<FQN DA BRONZE>';
```
(Substitua `catalog`/`schema` pelo que você usou neste notebook.)
